In [5]:
from pynotate import Project
import cv2
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
input_root_train = Path("c:/Users/cleme/OneDrive/Documents/postdoc/data/DNAI_Original/train")
input_root_test = Path("c:/Users/cleme/OneDrive/Documents/postdoc/data/DNAI_Original/test")

root_images_train = input_root_train / "images"
root_images_test = input_root_test / "images"

root_labels_train = input_root_train / "annotations"
root_labels_test = input_root_test / "annotations"
root_output = Path("c:/Users/cleme/OneDrive/Documents/postdoc/data/DNAI_Relabel")

list_images = sorted(list(root_images_train.glob("*.png"))) + sorted(list(root_images_test.glob("*.png"))) + sorted(list(root_images_test.glob("*.jpeg"))) + sorted(list(root_images_train.glob("*.jpeg")))
list_labels = sorted(list(root_labels_train.glob("*.png"))) + sorted(list(root_labels_test.glob("*.png"))) 

print(f"Found {len(list_images)} images and {len(list_labels)} labels")

Found 1032 images and 1032 labels


In [8]:
project = Project("DNAI_Relabel", input_dir=root_output / "data", output_dir=root_output / "annotations", is_segmentation=True,
                  segmentation_classes=["red", "green"])
with project as p:
    for img in tqdm(list_images):
        is_train = (root_images_train / img.name).exists()
        if (root_output / "data" / ("train/" if is_train else "test/") / img.name).exists():
            continue  # already done 
        if is_train:
            label_path = root_labels_train / img.name.replace(".jpeg", ".png")
        else:
            label_path = root_labels_test / img.name.replace(".jpeg", ".png")   
            
        image = cv2.imread(str(img), cv2.IMREAD_COLOR_RGB)
        label = cv2.imread(str(label_path), cv2.IMREAD_COLOR_RGB)
        red_mask = label[:, :, 0] > 200
        green_mask = label[:, :, 1] > 200
        img_name = "train/" + img.name if is_train else "test/" + img.name
        p.load_image(image=image, filename=img_name, copy_to_input_dir=True, segmentation_masks=[red_mask.astype(np.uint8), green_mask.astype(np.uint8)])

    

  0%|          | 0/1032 [00:00<?, ?it/s]

In [7]:
img.name

'01012.jpeg'